## COVID-19 vaccination rate vs. new cases rate

Data source: https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-County/8xkx-amqh

In [1]:
PROJECT_ID = '<< INSERT PROJET ID >>'

In [2]:
import pandas as pd

In [3]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [4]:
from google.cloud import bigquery

client = bigquery.Client(project=PROJECT_ID)

In [5]:
df_county_populations = client.query("""
SELECT 
country_name, 
subregion1_name, 
subregion2_name, 
MAX(population) as population
FROM 
`bigquery-public-data.covid19_open_data.covid19_open_data` 
WHERE country_name IN (
'United States of America')
AND aggregation_level = 2
GROUP BY 1, 2, 3
""").to_dataframe()

In [31]:
df_county_populations

,country_name,subregion1_name,subregion2_name,population
0,United States of America,Georgia,Appling County,18438
1,United States of America,Georgia,Atkinson County,8182
2,United States of America,Georgia,Bacon County,11205
3,United States of America,Georgia,Baker County,3159
4,United States of America,Georgia,Baldwin County,44963
...,...,...,...,...
3220,United States of America,Wyoming,Sweetwater County,43464
3221,United States of America,Wyoming,Teton County,23384
3222,United States of America,Wyoming,Uinta County,20431
3223,United States of America,Wyoming,Washakie County,8010


In [6]:
df_vax = pd.read_csv('https://data.cdc.gov/api/views/8xkx-amqh/rows.csv?accessType=DOWNLOAD')
df_vax.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (23,24,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date,FIPS,MMWR_week,Recip_County,Recip_State,Series_Complete_Pop_Pct,Series_Complete_Yes,Series_Complete_12Plus,Series_Complete_12PlusPop_Pct,Series_Complete_18Plus,Series_Complete_18PlusPop_Pct,Series_Complete_65Plus,Series_Complete_65PlusPop_Pct,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_12Plus,Administered_Dose1_Recip_12PlusPop_Pct,Administered_Dose1_Recip_18Plus,Administered_Dose1_Recip_18PlusPop_Pct,Administered_Dose1_Recip_65Plus,Administered_Dose1_Recip_65PlusPop_Pct,SVI_CTGY,Series_Complete_Pop_Pct_SVI,Series_Complete_12PlusPop_Pct_SVI,Series_Complete_18PlusPop_Pct_SVI,Series_Complete_65PlusPop_Pct_SVI
0,08/22/2021,18111,34,Newton County,IN,25.7,3598,3598.0,29.7,3497,31.6,1513,53.7,98.6,3932.0,28.1,3932.0,32.4,3799.0,34.3,1567.0,55.6,Low-Mod,Low VC/Low-Mod SVI,Low VC/Low-Mod SVI,Low-Mod VC/Low-Mod SVI,Mod-High VC/Low-Mod SVI
1,08/22/2021,40095,34,Marshall County,OK,33.6,5687,5687.0,39.5,5498,42.2,2329,62.9,93.6,7227.0,42.7,7227.0,50.1,6910.0,53.0,2758.0,74.5,High,Low-Mod VC/High SVI,Low-Mod VC/High SVI,Mod-High VC/High SVI,Mod-High VC/High SVI
2,08/22/2021,28137,34,Tate County,MS,30.5,8629,8629.0,35.6,8332,38.0,3190,65.6,98.3,10613.0,37.5,10610.0,43.7,10096.0,46.1,3523.0,72.4,High,Low-Mod VC/High SVI,Low-Mod VC/High SVI,Low-Mod VC/High SVI,Mod-High VC/High SVI
3,08/22/2021,29125,34,Maries County,MO,29.7,2583,2583.0,34.0,2500,36.0,1044,54.5,90.9,3032.0,34.9,3032.0,39.9,2906.0,41.9,1103.0,57.6,Mod-High,Low VC/Mod-High SVI,Low-Mod VC/Mod-High SVI,Low-Mod VC/Mod-High SVI,Mod-High VC/Mod-High SVI
4,08/22/2021,46137,34,Ziebach County,SD,20.6,569,569.0,24.4,524,26.2,123,46.6,83.8,655.0,23.8,NaN,0.0,NaN,0.0,NaN,0.0,High,Low VC/High SVI,Low VC/High SVI,Low VC/High SVI,Low-Mod VC/High SVI


In [7]:
county_lookups = {
    'Ventura County': 'Los Angeles',
    'San Bernardino County': 'Los Angeles',
    'Riverside County': 'Los Angeles',
    'Los Angeles County': 'Los Angeles',
    'Orange County': 'Los Angeles',
    'Alameda County': 'San Francisco',
    'Contra Costa County': 'San Francisco',
    'Marin County': 'San Francisco',
    'Napa County': 'San Francisco',
    'San Mateo County': 'San Francisco',
    'Santa Clara County': 'San Francisco',
    'Solano County': 'San Francisco',
    'Sonoma County': 'San Francisco',
    'San Francisco County': 'San Francisco',
    'Broward County': 'Miami',
    'Miami-Dade County': 'Miami',
    'Palm Beach County': 'Miami'
}

In [8]:
sorted(list(county_lookups.keys()))

['Alameda County',
 'Broward County',
 'Contra Costa County',
 'Los Angeles County',
 'Marin County',
 'Miami-Dade County',
 'Napa County',
 'Orange County',
 'Palm Beach County',
 'Riverside County',
 'San Bernardino County',
 'San Francisco County',
 'San Mateo County',
 'Santa Clara County',
 'Solano County',
 'Sonoma County',
 'Ventura County']

In [9]:
df_vax_filter = df_vax[df_vax['Recip_County'].isin(set(county_lookups.keys()))]
df_vax_filter = df_vax_filter[df_vax_filter['Recip_State'].isin({'CA', 'FL'})].copy()
df_vax_filter['city'] = df_vax_filter['Recip_County'].map(county_lookups)

In [10]:
df_vax_filter

,Date,FIPS,MMWR_week,Recip_County,Recip_State,Series_Complete_Pop_Pct,Series_Complete_Yes,Series_Complete_12Plus,Series_Complete_12PlusPop_Pct,Series_Complete_18Plus,Series_Complete_18PlusPop_Pct,Series_Complete_65Plus,Series_Complete_65PlusPop_Pct,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_12Plus,Administered_Dose1_Recip_12PlusPop_Pct,Administered_Dose1_Recip_18Plus,Administered_Dose1_Recip_18PlusPop_Pct,Administered_Dose1_Recip_65Plus,Administered_Dose1_Recip_65PlusPop_Pct,SVI_CTGY,Series_Complete_Pop_Pct_SVI,Series_Complete_12PlusPop_Pct_SVI,Series_Complete_18PlusPop_Pct_SVI,Series_Complete_65PlusPop_Pct_SVI,city
76,08/22/2021,12086,34,Miami-Dade County,FL,65.1,1767753,1767176.0,75.2,1685824,77.8,387461,85.6,98.6,2199293.0,80.9,2198172.0,93.6,2065773.0,95.3,463547.0,99.9,High,High VC/High SVI,High VC/High SVI,High VC/High SVI,High VC/High SVI,Miami
87,08/22/2021,06065,34,Riverside County,CA,44.1,1089814,1088078.0,52.5,1017230,54.8,270111,73.8,97.3,1288310.0,52.1,1285430.0,62.0,1192973.0,64.3,314475.0,85.9,High,Mod-High VC/High SVI,High VC/High SVI,High VC/High SVI,High VC/High SVI,Los Angeles
97,08/22/2021,12011,34,Broward County,FL,54.8,1070823,1070753.0,63.6,1012738,65.7,272431,81.4,98.6,1328124.0,68.0,1327553.0,78.9,1242550.0,80.6,325995.0,97.4,Mod-High,High VC/Mod-High SVI,High VC/Mod-High SVI,High VC/Mod-High SVI,High VC/Mod-High SVI,Miami
227,08/22/2021,12099,34,Palm Beach County,FL,52.9,791226,791182.0,60.3,757556,62.5,293354,80.5,98.6,936598.0,62.6,936289.0,71.4,888196.0,73.2,330472.0,90.7,Mod-High,High VC/Mod-High SVI,High VC/Mod-High SVI,High VC/Mod-High SVI,High VC/Mod-High SVI,Miami
350,08/22/2021,06071,34,San Bernardino County,CA,42.5,927249,925688.0,51.3,864354,53.7,190238,73.0,97.3,1095952.0,50.3,1093335.0,60.6,1014925.0,63.0,220061.0,84.5,High,Mod-High VC/High SVI,High VC/High SVI,High VC/High SVI,High VC/High SVI,Los Angeles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829816,12/13/2020,06055,51,Napa County,CA,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,Mod-High,NaN,NaN,NaN,NaN,San Francisco
829897,12/13/2020,12086,51,Miami-Dade County,FL,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,High,NaN,NaN,NaN,NaN,Miami
829915,12/13/2020,06059,51,Orange County,CA,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,Low-Mod,NaN,NaN,NaN,NaN,Los Angeles
829991,12/13/2020,06001,51,Alameda County,CA,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,Low-Mod,NaN,NaN,NaN,NaN,San Francisco


In [11]:
df_vax_filter[['Recip_County', 'city']].drop_duplicates()

,Recip_County,city
76,Miami-Dade County,Miami
87,Riverside County,Los Angeles
97,Broward County,Miami
227,Palm Beach County,Miami
350,San Bernardino County,Los Angeles
434,Orange County,Los Angeles
958,Contra Costa County,San Francisco
1109,Santa Clara County,San Francisco
1151,Napa County,San Francisco
1167,Alameda County,San Francisco


In [12]:
## Join population totals
df_vax_filter_w_populations = pd.merge(df_vax_filter,
                                       df_county_populations,
                                       left_on='Recip_County',
                                       right_on='subregion2_name')

In [13]:
df_vax_filter_w_populations.head()

,Date,FIPS,MMWR_week,Recip_County,Recip_State,Series_Complete_Pop_Pct,Series_Complete_Yes,Series_Complete_12Plus,Series_Complete_12PlusPop_Pct,Series_Complete_18Plus,Series_Complete_18PlusPop_Pct,Series_Complete_65Plus,Series_Complete_65PlusPop_Pct,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_12Plus,Administered_Dose1_Recip_12PlusPop_Pct,Administered_Dose1_Recip_18Plus,Administered_Dose1_Recip_18PlusPop_Pct,Administered_Dose1_Recip_65Plus,Administered_Dose1_Recip_65PlusPop_Pct,SVI_CTGY,Series_Complete_Pop_Pct_SVI,Series_Complete_12PlusPop_Pct_SVI,Series_Complete_18PlusPop_Pct_SVI,Series_Complete_65PlusPop_Pct_SVI,city,country_name,subregion1_name,subregion2_name,population
0,08/22/2021,12086,34,Miami-Dade County,FL,65.1,1767753,1767176.0,75.2,1685824,77.8,387461,85.6,98.6,2199293.0,80.9,2198172.0,93.6,2065773.0,95.3,463547.0,99.9,High,High VC/High SVI,High VC/High SVI,High VC/High SVI,High VC/High SVI,Miami,United States of America,Florida,Miami-Dade County,2713295
1,08/21/2021,12086,33,Miami-Dade County,FL,64.7,1756630,1756054.0,74.7,1676824,77.4,386548,85.4,98.6,2189518.0,80.6,2188400.0,93.1,2057947.0,95.0,462726.0,99.9,High,High VC/High SVI,High VC/High SVI,High VC/High SVI,High VC/High SVI,Miami,United States of America,Florida,Miami-Dade County,2713295
2,08/20/2021,12086,33,Miami-Dade County,FL,64.4,1750009,1749433.0,74.5,1671460,77.1,386071,85.3,98.6,2182521.0,80.3,2181404.0,92.8,2052368.0,94.7,462192.0,99.9,High,High VC/High SVI,High VC/High SVI,High VC/High SVI,High VC/High SVI,Miami,United States of America,Florida,Miami-Dade County,2713295
3,08/19/2021,12086,33,Miami-Dade County,FL,64.2,1743306,1742731.0,74.2,1665880,76.9,385452,85.2,98.6,2174427.0,80.0,2173313.0,92.5,2045810.0,94.4,461518.0,99.9,High,High VC/High SVI,High VC/High SVI,High VC/High SVI,High VC/High SVI,Miami,United States of America,Florida,Miami-Dade County,2713295
4,08/18/2021,12086,33,Miami-Dade County,FL,63.9,1737309,1736734.0,73.9,1660997,76.6,384960,85.1,98.6,2167533.0,79.8,2166419.0,92.2,2040268.0,94.1,460941.0,99.9,High,High VC/High SVI,High VC/High SVI,High VC/High SVI,High VC/High SVI,Miami,United States of America,Florida,Miami-Dade County,2713295


In [14]:
df_vax_filter_w_populations.to_csv('df_vax_filter.csv', index=False)

In [15]:
us_counties = ',\n'.join([f"'{x}'" for x in sorted(county_lookups.keys())])
print(us_counties)

'Alameda County',
'Broward County',
'Contra Costa County',
'Los Angeles County',
'Marin County',
'Miami-Dade County',
'Napa County',
'Orange County',
'Palm Beach County',
'Riverside County',
'San Bernardino County',
'San Francisco County',
'San Mateo County',
'Santa Clara County',
'Solano County',
'Sonoma County',
'Ventura County'


In [16]:
## Get covid cases for populations under investigation

query = f"""
SELECT 
date,
country_name, 
subregion1_name, 
subregion2_name, 
population,
new_confirmed,
new_deceased,
new_persons_vaccinated,
new_persons_fully_vaccinated,
cumulative_persons_vaccinated,
cumulative_persons_fully_vaccinated,
cumulative_tested
FROM 
`bigquery-public-data.covid19_open_data.covid19_open_data` 
WHERE (country_name IN ('Australia')
AND subregion1_name IN ('New South Wales', 'Victoria'))
OR (country_name IN ('United Kingdom')
AND subregion1_name IN ('London Region'))
OR
(country_name = 'United States of America' 
AND subregion1_name IN (
    'California',
    'Florida'
)
AND subregion2_name IN ({us_counties})
)
"""

print(query)

df_covid_cases = client.query(query).to_dataframe()


SELECT 
date,
country_name, 
subregion1_name, 
subregion2_name, 
population,
new_confirmed,
new_deceased,
new_persons_vaccinated,
new_persons_fully_vaccinated,
cumulative_persons_vaccinated,
cumulative_persons_fully_vaccinated,
cumulative_tested
FROM 
`bigquery-public-data.covid19_open_data.covid19_open_data` 
WHERE (country_name IN ('Australia')
AND subregion1_name IN ('New South Wales', 'Victoria'))
OR (country_name IN ('United Kingdom')
AND subregion1_name IN ('London Region'))
OR
(country_name = 'United States of America' 
AND subregion1_name IN (
    'California',
    'Florida'
)
AND subregion2_name IN ('Alameda County',
'Broward County',
'Contra Costa County',
'Los Angeles County',
'Marin County',
'Miami-Dade County',
'Napa County',
'Orange County',
'Palm Beach County',
'Riverside County',
'San Bernardino County',
'San Francisco County',
'San Mateo County',
'Santa Clara County',
'Solano County',
'Sonoma County',
'Ventura County')
)



In [27]:
def classify_region(row):
  if row.subregion2_name:
    return county_lookups.get(row.subregion2_name)
  return row.subregion1_name

df_covid_cases['region'] = df_covid_cases.apply(classify_region, axis=1)

In [28]:
df_covid_cases.groupby('region').agg({
    'date': 'count'
})

,date
region,
London Region,602
Los Angeles,3612
Miami,1806
New South Wales,602
San Francisco,5418
Victoria,602


In [38]:
## Get population per region
df_region_population = df_covid_cases[['region', 'subregion1_name', 'population']].drop_duplicates().groupby('region').agg({
    'population': 'sum'
}).reset_index()

df_region_population.style.format({'population': '{:,.0f}'})

,region,population
0,London Region,"8,982,256"
1,Los Angeles,"20,048,874"
2,Miami,"6,118,155"
3,New South Wales,"7,480,228"
4,San Francisco,"7,730,789"
5,Victoria,"5,926,624"


In [39]:
df_covid_cases_daily = df_covid_cases.groupby(['date', 'region']).agg({
    'new_confirmed': 'sum',
    'cumulative_persons_vaccinated': 'sum',
    'cumulative_persons_fully_vaccinated': 'sum'
}).reset_index()

df_covid_cases_daily = pd.merge(df_covid_cases_daily,
                                df_region_population,
                                on='region')

In [40]:
df_covid_cases_daily

,date,region,new_confirmed,cumulative_persons_vaccinated,cumulative_persons_fully_vaccinated,population
0,2020-01-01,London Region,0.0,0.0,0.0,8982256
1,2020-01-02,London Region,0.0,0.0,0.0,8982256
2,2020-01-03,London Region,0.0,0.0,0.0,8982256
3,2020-01-04,London Region,0.0,0.0,0.0,8982256
4,2020-01-05,London Region,0.0,0.0,0.0,8982256
...,...,...,...,...,...,...
3607,2021-08-20,Victoria,57.0,2715401.0,1549472.0,5926624
3608,2021-08-21,Victoria,60.0,2747285.0,1588286.0,5926624
3609,2021-08-22,Victoria,66.0,2770971.0,1614057.0,5926624
3610,2021-08-23,Victoria,0.0,2790386.0,1629776.0,5926624


In [41]:
df_covid_cases_daily.to_csv('df_covid_cases_daily.csv', index=False)